In [133]:
import pandas as pd
import numpy as np
import re
import math

In [3]:
batting_df = pd.read_csv('batting_data_for_world_cup.csv')
bowling_df = pd.read_csv('bowling_data_for_world_cup.csv')

In [5]:
batting_df.shape

(1816, 11)

In [14]:
def clean_string(s):
    s = re.sub(r'[^\x00-\x7F]+', '', s)  # Remove non-ASCII characters
    s = re.sub(r'\(c\)|\(†\)', '', s)   # Remove (c) or (†)
    return s

In [15]:
batting_df['Batsman Name'] = batting_df['Batsman Name'].apply(lambda x:clean_string(x))

In [21]:
batting_df['Dismissal'] = batting_df['Dismissal'].apply(lambda x:0 if x=='not out' else 1)

In [189]:
batting_df['Hundreds'] = batting_df['Runs'].apply(lambda x:1 if x >=100 else 0)

In [191]:
batting_df['Fifties'] = batting_df['Runs'].apply(lambda x:1 if x>=50 and x<100 else 0)

In [192]:
batting_df.head()

,Match Name,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,4s,6s,Match id,Strike Rate,Hundreds,Fifties
0,West Indies VsIreland,West Indies,1,Shai Hope,1,29,44,3,0,4339,65.90,0,0
1,West Indies VsIreland,West Indies,2,Justin Greaves,1,7,24,1,0,4339,29.16,0,0
2,West Indies VsIreland,West Indies,3,Nicholas Pooran,1,13,34,2,0,4339,38.23,0,0
3,West Indies VsIreland,West Indies,4,Shamarh Brooks,1,93,89,9,3,4339,104.49,0,1
4,West Indies VsIreland,West Indies,5,Roston Chase,1,1,4,0,0,4339,25.00,0,0


In [63]:
batting_df['Balls'] = batting_df['Balls'].fillna(0)
batting_df['Runs'] = batting_df['Runs'].fillna(0)
batting_df['4s'] = batting_df['4s'].fillna(0)
batting_df['6s'] = batting_df['6s'].fillna(0)

In [64]:
# batting_df['Strike Rate'] = batting_df['Strike Rate'].astype(float)
batting_df['Strike Rate'] = batting_df['Strike Rate'].replace('-', 0.0).astype(float)
batting_df['Runs'] = batting_df['Runs'].replace('-', 0).astype(int)
batting_df['Balls'] = batting_df['Balls'].replace('-', 0).astype(int)
batting_df['4s'] = batting_df['4s'].replace('-', 0).astype(int)
batting_df['6s'] = batting_df['6s'].replace('-', 0).astype(int)

In [65]:
batting_df['Runs'] = batting_df['Runs'].astype(int)
batting_df['Balls'] = batting_df['Balls'].astype(int)
batting_df['Dismissal'] = batting_df['Dismissal'].astype(int)
batting_df['4s'] = batting_df['4s'].astype(int)
batting_df['6s'] = batting_df['6s'].astype(int)
batting_df['Match id'] = batting_df['Match id'].astype(int)
batting_df['Strike Rate'] = batting_df['Strike Rate'].astype(float)

In [194]:
grouped_batting_stats = batting_df.groupby(['Batsman Name','teamInnings']).agg(
    Runs_Scored=('Runs', 'sum'),
    Fours_Scored=('4s', 'sum'),
    Sixes_scored=('6s', 'sum'),
    Balls_Faced=('Balls', 'sum'),
    Total_Matches=('Match id', 'nunique'),
    Total_dismissals = ('Dismissal','sum'),
    Total_hundreds = ('Hundreds','sum'),
    Total_fifties = ('Fifties','sum')
).reset_index()

In [195]:
grouped_batting_stats = grouped_batting_stats.sort_values(by = "Batsman Name")

In [197]:
grouped_batting_stats['Strike Rate'] = (grouped_batting_stats['Runs_Scored']/grouped_batting_stats['Balls_Faced'])*100
grouped_batting_stats['Strike Rate'] = grouped_batting_stats['Strike Rate'].round(2)

In [198]:
grouped_batting_stats['Batting Average'] = grouped_batting_stats['Runs_Scored']/grouped_batting_stats['Total_dismissals']

In [201]:
grouped_batting_stats.head()

,Batsman Name,teamInnings,Runs_Scored,Fours_Scored,Sixes_scored,Balls_Faced,Total_Matches,Total_dismissals,Total_hundreds,Total_fifties,Strike Rate,Batting Average
0,Aaron Finch,Australia,174,14,3,245,14,14,0,1,71.02,12.428571
1,Adam Zampa,Australia,40,4,0,66,8,2,0,0,60.61,20.000000
2,Adil Rashid,England,71,4,1,94,9,3,0,0,75.53,23.666667
3,Afif Hossain,Bangladesh,396,40,5,488,15,12,0,3,81.15,33.000000
4,Agha Salman,Pakistan,83,6,2,75,3,2,0,0,110.67,41.500000


In [200]:
bowling_df['Bowler Name'] = bowling_df['Bowler Name'].apply(lambda x:clean_string(x))

In [104]:
bowling_df = bowling_df.drop(columns=['wides','no balls','dots','Maiden'])

In [113]:
bowling_df.head()

,match,teamInnings,Bowler Name,Over,Runs,Wickets,Economy,fours,sixes,Match id
0,West Indies VsIreland,Ireland,Josh Little,10.0,46,1,4.60,4,1,4339
1,West Indies VsIreland,Ireland,Mark Adair,8.5,38,3,4.30,4,1,4339
2,West Indies VsIreland,Ireland,Craig Young,9.0,56,3,6.22,8,0,4339
3,West Indies VsIreland,Ireland,Curtis Campher,10.0,47,1,4.70,2,2,4339
4,West Indies VsIreland,Ireland,Andy McBrine,8.0,52,2,6.50,3,4,4339


In [121]:
bowling_df['Runs'] = bowling_df['Runs'].astype(int)
bowling_df['Wickets'] = bowling_df['Wickets'].astype(int)
bowling_df['fours'] = bowling_df['fours'].astype(int)
bowling_df['sixes'] = bowling_df['sixes'].astype(int)

In [118]:
grouped_bowling_stats = bowling_df.groupby(['Bowler Name','teamInnings']).agg(
    Runs_Conceded=('Runs','sum'),
    Overs_Bowled=('Over','sum'),
    Total_Wickets_Taken=('Wickets','sum'),
    Fours_Conceded=('fours','sum'),
    Sixes_Conceded=('sixes','sum'),
    Total_Matches=('Match id', 'nunique'),
).reset_index()

In [126]:
grouped_bowling_stats['Economy'] = grouped_bowling_stats['Runs_Conceded']/grouped_bowling_stats['Overs_Bowled']
grouped_bowling_stats['Economy'] = grouped_bowling_stats['Economy'].round(1)

In [167]:
def calculate_balls_bowled(overs):
    integer_part = int(overs)
    decimal_part = overs - integer_part
    total_balls = int(integer_part * 6 + (10*decimal_part))
    return total_balls


In [168]:
grouped_bowling_stats['Total_Balls_Bowled'] = grouped_bowling_stats['Overs_Bowled'].apply(lambda x:calculate_balls_bowled(x))

In [172]:
grouped_bowling_stats['Bowling Average'] = grouped_bowling_stats['Runs_Conceded']/grouped_bowling_stats['Total_Wickets_Taken']
grouped_bowling_stats['Bowling Average'] = grouped_bowling_stats['Bowling Average'].round(2)

In [178]:
grouped_bowling_stats['Bowling Strike Rate'] = grouped_bowling_stats['Total_Balls_Bowled']/grouped_bowling_stats['Total_Wickets_Taken']
grouped_bowling_stats['Bowling Strike Rate'] = grouped_bowling_stats['Bowling Strike Rate'].round(2)

In [202]:
grouped_batting_stats['Batting Average'] =  grouped_batting_stats['Batting Average'].round(2)

In [204]:
grouped_batting_stats = grouped_batting_stats.drop(columns=['Fours_Scored','Sixes_scored','Balls_Faced','Total_Matches','Total_dismissals'])

In [208]:
grouped_batting_stats = grouped_batting_stats.rename(columns={'Batsman Name':'Player_name','teamInnings':'Team Name'})

In [210]:
grouped_bowling_stats = grouped_bowling_stats.rename(columns={'Bowler Name':'Player_name','teamInnings':'Team Name'})

In [182]:
grouped_bowling_stats = grouped_bowling_stats.drop(columns=['Overs_Bowled','Fours_Conceded','Sixes_Conceded','Total_Matches','Total_Balls_Bowled'],axis='columns')

In [209]:
grouped_batting_stats.head()

,Player_name,Team Name,Runs_Scored,Total_hundreds,Total_fifties,Strike Rate,Batting Average
0,Aaron Finch,Australia,174,0,1,71.02,12.43
1,Adam Zampa,Australia,40,0,0,60.61,20.00
2,Adil Rashid,England,71,0,0,75.53,23.67
3,Afif Hossain,Bangladesh,396,0,3,81.15,33.00
4,Agha Salman,Pakistan,83,0,0,110.67,41.50


In [211]:
grouped_bowling_stats.head()

,Player_name,Team Name,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate
0,Adam Milne,New Zealand,67,1,6.7,67.00,60.00
1,Adam Zampa,Australia,614,34,5.0,18.06,21.65
2,Adil Rashid,England,457,20,5.7,22.85,24.00
3,Afif Hossain,Bangladesh,45,2,4.3,22.50,32.00
4,Agha Salman,Pakistan,88,2,4.9,44.00,54.00


In [225]:
merged_df = pd.merge(grouped_batting_stats,grouped_bowling_stats,on='Player_name',how='outer')
merged_df.head()

,Player_name,Team Name_x,Runs_Scored,Total_hundreds,Total_fifties,Strike Rate,Batting Average,Team Name_y,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate
0,Aaron Finch,Australia,174.0,0.0,1.0,71.02,12.43,NaN,NaN,NaN,NaN,NaN,NaN
1,Adam Zampa,Australia,40.0,0.0,0.0,60.61,20.00,Australia,614.0,34.0,5.0,18.06,21.65
2,Adil Rashid,England,71.0,0.0,0.0,75.53,23.67,England,457.0,20.0,5.7,22.85,24.00
3,Afif Hossain,Bangladesh,396.0,0.0,3.0,81.15,33.00,Bangladesh,45.0,2.0,4.3,22.50,32.00
4,Agha Salman,Pakistan,83.0,0.0,0.0,110.67,41.50,Pakistan,88.0,2.0,4.9,44.00,54.00


In [227]:
merged_df = merged_df.fillna(0)

In [228]:
merged_df = merged_df.rename(columns={'Team Name_x':'Team Name'})
merged_df = merged_df.drop(columns = ['Team Name_y'])

In [229]:
merged_df.head()

,Player_name,Team Name,Runs_Scored,Total_hundreds,Total_fifties,Strike Rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate
0,Aaron Finch,Australia,174.0,0.0,1.0,71.02,12.43,0.0,0.0,0.0,0.00,0.00
1,Adam Zampa,Australia,40.0,0.0,0.0,60.61,20.00,614.0,34.0,5.0,18.06,21.65
2,Adil Rashid,England,71.0,0.0,0.0,75.53,23.67,457.0,20.0,5.7,22.85,24.00
3,Afif Hossain,Bangladesh,396.0,0.0,3.0,81.15,33.00,45.0,2.0,4.3,22.50,32.00
4,Agha Salman,Pakistan,83.0,0.0,0.0,110.67,41.50,88.0,2.0,4.9,44.00,54.00


In [231]:
merged_df.to_csv('players_stats_for_world_cup_model.csv',index=False)